In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

In [6]:
# Config
batch_size = 64
epochs = 30
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Dataset

In [4]:
# Datasets
train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

# DataLoaders
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [5]:
# Data 확인
for x, y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


### Naive ConvNet

In [7]:
class Naive_ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.convlayers = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(7, 1)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = self.convlayers(x)
        x = self.classifier(x)
        return x

In [8]:
naive_convnet = Naive_ConvNet().to(device)
print(naive_convnet)

Naive_ConvNet(
  (convlayers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=7, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


### Train

In [12]:
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(naive_convnet.parameters(), lr=learning_rate)

def train_one_epoch(model, optimizer, data_loader, device):
    model.train()
    one_epoch_loss = 0
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        pred = model(x)
        loss = criterian(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        one_epoch_loss += loss.item()
    return one_epoch_loss / len(data_loader)

def eval_one_epoch(model, data_loader, device):
    model.eval()
    one_epoch_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)

            pred = model(x)
            loss = criterian(pred, y)

            one_epoch_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    return one_epoch_loss / len(data_loader), correct / len(data_loader.dataset)

def train():
    for epoch in range(epochs):
        train_loss = train_one_epoch(naive_convnet, optimizer, train_dataloader, device)
        print(f'Epoch {epoch+1} / {epochs}')
        print(f'Train Loss: {train_loss:.4f}')
        eval_loss, correct = eval_one_epoch(naive_convnet, test_dataloader, device)
        print(f'Eval Loss: {eval_loss:.4f}, Eval Acc: {100 * correct:.4f}%')
train()

Epoch 1 / 30
Train Loss: 0.1954
Eval Loss: 0.2444, Eval Acc: 91.4600%


KeyboardInterrupt: 